# Import Library

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow import keras

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import pickle
from pandas.tseries.offsets import DateOffset

from database import export_db

# Get Datast

In [2]:
btc_df = export_db('BTC')
train_dates = btc_df["OpenTime"]
train = btc_df['Open']

Exporting BTC


# Scale Data

In [3]:
train = train.values.reshape(-1, 1)
scaler = StandardScaler() 
train_transformed = scaler.fit_transform(train)
train_transformed

array([[-0.88104107],
       [-0.87830276],
       [-0.8770617 ],
       ...,
       [ 0.02475432],
       [ 0.0256507 ],
       [ 0.02451374]])

# Train Test Spli

In [6]:
train_size = int(len(train_transformed) * 0.66)
train, test = train_transformed[:train_size, :], train_transformed[train_size:, :]

In [8]:
def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
        #print(i)
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])
        
    return np.array(x),np.array(y)
    

seq_size = 100  # Number of time steps to look back 
#Larger sequences (look further back) may improve forecasting.

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)

In [11]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# 

In [12]:
trainX.shape

(28921, 1, 100)

# model

In [18]:
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
history = model.fit_generator(generator,epochs=100,verbose=1)

/var/folders/dw/94thms1n5817tsrqwqcs_3rr0000gn/T/ipykernel_48830/1858071613.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator,epochs=100,verbose=1)


Epoch 1/100
4835/4835 [==============================] - 56s 11ms/step - loss: 0.0025
Epoch 2/100
4835/4835 [==============================] - 55s 11ms/step - loss: 7.4386e-04
Epoch 3/100
4835/4835 [==============================] - 51s 11ms/step - loss: 4.8898e-04
Epoch 4/100
4835/4835 [==============================] - 58s 12ms/step - loss: 4.1859e-04
Epoch 5/100
4835/4835 [==============================] - 59s 12ms/step - loss: 3.7208e-04
Epoch 6/100
4835/4835 [==============================] - 60s 12ms/step - loss: 3.7361e-04
Epoch 7/100
4835/4835 [==============================] - 66s 14ms/step - loss: 3.6391e-04
Epoch 8/100
4835/4835 [==============================] - 81s 17ms/step - loss: 3.4480e-04
Epoch 9/100
4835/4835 [==============================] - 88s 18ms/step - loss: 3.3954e-04
Epoch 10/100
4835/4835 [==============================] - 96s 20ms/step - loss: 3.3114e-04
Epoch 11/100
4835/4835 [==============================] - 92s 19ms/step - loss: 3.2246e-04
Epoch 12/100

In [19]:
with open("model.pickle", "wb") as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://a96f1843-e239-492b-973f-245403ea873c/assets


In [16]:
pred_list = []
n_features = 1
n_input = 100
batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
add_dates = [train_dates.index[-1] + DateOffset(hours=x) for x in range(0,20) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=train_dates.columns)
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

df_proj = pd.concat([train_dates,df_predict], axis=1)

df_proj.tail(12)

ValueError: in user code:

    File "/Users/prozapple/.pyenv/versions/anaconda3-2021.11/envs/project3-model/lib/python3.10/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/prozapple/.pyenv/versions/anaconda3-2021.11/envs/project3-model/lib/python3.10/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/prozapple/.pyenv/versions/anaconda3-2021.11/envs/project3-model/lib/python3.10/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/prozapple/.pyenv/versions/anaconda3-2021.11/envs/project3-model/lib/python3.10/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/prozapple/.pyenv/versions/anaconda3-2021.11/envs/project3-model/lib/python3.10/site-packa

# REST

In [ ]:
trainX = []
trainY = []

n_future = 100
n_past = 1000 

for i in range(n_past, len(train_transformed) - n_future +1):
    trainX.append(train_transformed[i - n_past:i, 0:train_transformed.shape[1]])
    trainY.append(train_transformed[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()


# fit the model
history = model.fit(trainX, trainY, epochs=5, batch_size=16, validation_split=0.1, verbose=1)

In [ ]:
import pickle
with open("2layer.pickle", "rb") as f:
    model = pickle.load(f)

In [ ]:
# make predictions
n_future = 90
forecast_period_dates = pd.date_range(list(train_dates)[-1], periods=n_future, freq="1h").tolist()
forecast = model.predict(trainX[-n_future:])

In [ ]:
y_pred_future = scaler.inverse_transform(forecast)
y_pred_future

In [ ]:
testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))